In [48]:
import numpy as np
import tensorflow as tf

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave, imshow_collection
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [49]:
# Load image

image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype=float)
#image[:,:,0]
#image.shape # -> (400, 400, 3)

## Prepare image

**numpy.reshape**

In [50]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128 # Normalize lab values (-128 to 128) to values between -1 and 1.

# Add forth dimension - Why?
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [67]:
# Building the neural network
model = Sequential()
## Encoder
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
## Decoder
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.compile(optimizer='rmsprop', loss='mse')

# save initial weights for batch training different epochs
model.save_weights('initial_model.h5')

model.fit - Trains the model for a fixed number of epochs (iteratrion on a dataset).

model.evaluate - Returns the loss value & metric value for the model in test mode.

model.predict - Generates output prediction for the input samples

In [69]:
# Train model -> Epochs: 10, 100, 500, 1000, 2000, 3000
training_epochs = [10, 100, 500, 1000, 2000, 3000]
for epochs in training_epochs:
    model.load_weights('initial_model.h5')
    model.fit(x=X, y=Y, batch_size=1, epochs=epochs)
    
    model.evaluate(X, Y, batch_size=1)
    output = model.predict(X)
    output *= 128

    canvas = np.zeros((400, 400, 3))
    canvas[:,:,0] = X[0][:,:,0]
    canvas[:,:,1:] = output[0]

    imsave(f"woman_result_e{epochs}.jpg", lab2rgb(canvas))
    imsave("woman_gray.jpg", rgb2gray(lab2rgb(canvas)))

00
1/1 [==============================] - 0s 108ms/sample - loss: 8.1521e-05
Epoch 2783/3000
1/1 [==============================] - 0s 102ms/sample - loss: 8.5236e-05
Epoch 2784/3000
1/1 [==============================] - 0s 106ms/sample - loss: 9.2828e-05
Epoch 2785/3000
1/1 [==============================] - 0s 109ms/sample - loss: 1.0269e-04
Epoch 2786/3000
1/1 [==============================] - 0s 105ms/sample - loss: 1.1553e-04
Epoch 2787/3000
1/1 [==============================] - 0s 105ms/sample - loss: 1.1341e-04
Epoch 2788/3000
1/1 [==============================] - 0s 108ms/sample - loss: 1.1648e-04
Epoch 2789/3000
1/1 [==============================] - 0s 104ms/sample - loss: 1.0379e-04
Epoch 2790/3000
1/1 [==============================] - 0s 104ms/sample - loss: 9.2037e-05
Epoch 2791/3000
1/1 [==============================] - 0s 104ms/sample - loss: 8.4838e-05
Epoch 2792/3000
1/1 [==============================] - 0s 105ms/sample - loss: 1.0197e-04
Epoch 2793/3000
1/1 [==